# Note

#### 1. use kernel_initializer = 'ones' instead of 'zeros' -> the gradients do not flow through the ReLu layer

In [1]:
import numpy as np

In [2]:
# create dummy files
np.random.seed(42)
EMBEDDING_SIZE = 15
X = np.random.randint(2, size=(1000,100))
X_val = np.random.randint(2, size=(1000,100))
y = np.random.randint(2, size=1000)
y_val = np.random.randint(2, size=1000)

In [3]:
X.shape

(1000, 100)

In [4]:
y.shape

(1000,)

In [5]:
from keras.layers import Input, Dense, TimeDistributed, Embedding
from keras.layers import Concatenate, Reshape, Lambda, Multiply, multiply, concatenate
from keras.models import Model
from keras import backend as K
import tensorflow as tf

C:\Users\Anneke\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
y_tanh = []
y_tanh_val = []

for i in y:
    if i==0:
        y_tanh.append(-1)
    else:
        y_tanh.append(1)


for i in y_val:
    if i==0:
        y_tanh_val.append(-1)
    else:
        y_tanh_val.append(1)
        
y_tanh = np.asarray(y_tanh)  
y_tanh_val = np.asarray(y_tanh_val)

In [7]:
# make first model

def build_base_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    tanh_output = Dense(1, activation='tanh', name='tanh_output')(input_layer)
    
    model = Model(inputs=input_layer, outputs=tanh_output)
    model.summary()
    return model

base_model = build_base_model(X.shape[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 101       
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Combined model

combined_input_layer = Input(shape=(X.shape[1],))

ht_1 = Input(shape=(1,))
ht_2 = Input(shape=(1,))

label_layer = base_model(combined_input_layer)

dense_1 = Dense(1, activation='relu')(Multiply()([ht_1, label_layer]))
dense_2 = Dense(1, activation='relu')(Multiply()([ht_2, label_layer]))

concat = Concatenate()([dense_1, dense_2])

output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_1, ht_2], outputs=output_layer)
combined_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            101         input_2[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
multiply_1

In [9]:
base_model.compile(loss='mse',
            optimizer='adam',
            metrics=['acc'])

In [10]:
combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [11]:
# make second model
# first, make the vector to see if it is appears 

# we have two terms (human-terms) for now. c1 == pos word, c2 == neg word
# 1st column gives the feature index where that particular human_terms located
# 2nd column gives whether it is positive/negative word
term = [[1,1],[2,-1]]
term = np.asarray(term)

#assume X is a document vector [1,n] where n is the number of terms

def np_present_by_term(X, term):
    np_vector = np.zeros([term.shape[0],1])
    
    for i,(feature_index,np_term) in enumerate(term):
        if X[feature_index] == 1:
            np_vector[i] = np_term
    
    return np_vector

# Assuming 'term' is a [1X2] vector
def np_present_by_doc(X, term):
    np_vector = np.zeros([X.shape[0],1])
    
    for i, x in enumerate(X):
        if x[term[0]] == 1:
            np_vector[i] = term[1]
    
    return np_vector

In [12]:
ht_1_input = np_present_by_doc(X, term[0])
ht_2_input = np_present_by_doc(X, term[1])

ht_1_input_val = np_present_by_doc(X_val, term[0])
ht_2_input_val = np_present_by_doc(X_val, term[1])

In [13]:
base_model.fit(X, y_tanh, validation_data=(X_val, y_tanh_val), epochs=10, verbose=1)

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 11s 11ms/step - loss: 1.2658 - acc: 0.2460 - val_loss: 1.2741 - val_acc: 0.2300
Epoch 2/10
1000/1000 [==============================] - 0s 233us/step - loss: 1.2297 - acc: 0.2530 - val_loss: 1.2626 - val_acc: 0.2260
Epoch 3/10
1000/1000 [==============================] - 0s 201us/step - loss: 1.2123 - acc: 0.2490 - val_loss: 1.2414 - val_acc: 0.2110
Epoch 4/10
1000/1000 [==============================] - 0s 234us/step - loss: 1.1941 - acc: 0.2370 - val_loss: 1.2583 - val_acc: 0.2170
Epoch 5/10
1000/1000 [==============================] - 0s 214us/step - loss: 1.1793 - acc: 0.2440 - val_loss: 1.2443 - val_acc: 0.2110
Epoch 6/10
1000/1000 [==============================] - 0s 204us/step - loss: 1.1619 - acc: 0.2400 - val_loss: 1.2324 - val_acc: 0.1990
Epoch 7/10
1000/1000 [==============================] - 0s 232us/step - loss: 1.1469 - acc: 0.2280 - val_loss: 1.2268 - val_acc: 0.1970


In [14]:
combined_model.fit([X, ht_1_input, ht_2_input], y, 
                   validation_data=([X_val, ht_1_input_val, ht_2_input_val], y_val), 
                   epochs=10, verbose=1)

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 667us/step - loss: 0.2499 - acc: 0.5070 - val_loss: 0.2500 - val_acc: 0.4830
Epoch 2/10
1000/1000 [==============================] - 0s 334us/step - loss: 0.2496 - acc: 0.5420 - val_loss: 0.2502 - val_acc: 0.4800
Epoch 3/10
1000/1000 [==============================] - 0s 318us/step - loss: 0.2493 - acc: 0.5380 - val_loss: 0.2505 - val_acc: 0.4710
Epoch 4/10
1000/1000 [==============================] - 0s 334us/step - loss: 0.2491 - acc: 0.5380 - val_loss: 0.2508 - val_acc: 0.4710
Epoch 5/10
1000/1000 [==============================] - 0s 318us/step - loss: 0.2488 - acc: 0.5340 - val_loss: 0.2510 - val_acc: 0.4700
Epoch 6/10
1000/1000 [==============================] - 0s 332us/step - loss: 0.2485 - acc: 0.5400 - val_loss: 0.2511 - val_acc: 0.4740
Epoch 7/10
1000/1000 [==============================] - 0s 327us/step - loss: 0.2482 - acc: 0.5440 - val_loss: 0.2513 - val_acc: 0.4700


### Simple Domain Test

In [15]:
# Generate 16 documents
# n1 = negative word with importance 1
# p2 = positive word with importance 2
# p3 = positive word with importance 3
# n4 = negative word with importance 4
# movie = neutral word with importance 0

word_list = ['movie', 'bad', 'good', 'amazing', 'awful']
word_weight = np.asarray([0, -1, 2, 3, -4], dtype='int32')
word_weight.shape

(5,)

In [16]:
from itertools import combinations

num = 16
data = np.zeros([num, len(word_list)])
index=0
label = []

for i in range(5):
    # get the combination
    perm = combinations([1,2,3,4], i)
    
    for j in list(perm):
        # always put neutral word
        data[index, 0] = 1
        
        # if its only 1 terms
        if i==1:
            data[index,j] = 1
        else:
            for k in j:
                data[index, k] = 1
        
        # calculate weights
        
        weights = data[index,:] * word_weight
        label.append(np.sign(np.sum(weights)))
        
        index+=1
        
label = np.asarray(label)

In [17]:
np_label = (label == 1).astype('int32')
np_label

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0])

In [18]:
label

array([ 0., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,
       -1.,  1.,  0.])

In [19]:
data.shape[0]

16

In [21]:
# build the network

base_model = build_base_model(data.shape[1])

# build second model

# Combined model

combined_input_layer = Input(shape=(data.shape[1],))

# build the hard coded weight for human terms
ht_1 = Input(shape=(1,))
ht_2 = Input(shape=(1,))
ht_3 = Input(shape=(1,))
ht_4 = Input(shape=(1,))

label_layer = base_model(combined_input_layer)

# multiply and pass it into relu
dense_1 = Dense(1, activation='relu', name='dense_1', use_bias=False, kernel_initializer='ones')(Multiply(name='multiply_1')([ht_1, label_layer]))
dense_2 = Dense(1, activation='relu', name='dense_2', use_bias=False, kernel_initializer='ones')(Multiply(name='multiply_2')([ht_2, label_layer]))
dense_3 = Dense(1, activation='relu', name='dense_3', use_bias=False, kernel_initializer='ones')(Multiply(name='multiply_3')([ht_3, label_layer]))
dense_4 = Dense(1, activation='relu', name='dense_4', use_bias=False, kernel_initializer='ones')(Multiply(name='multiply_4')([ht_4, label_layer]))

# concat all the result
concat = Concatenate(name='concatenate')([dense_1, dense_2, dense_3, dense_4])

output_layer = Dense(1, activation='sigmoid')(concat)


combined_model = Model(inputs=[combined_input_layer, ht_1, ht_2, ht_3, ht_4], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 5)                 0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 5)            0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 1)            0                                            
_________________________________________________

In [22]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [23]:
def np_present_by_doc(X, index, weight):
    np_vector = np.zeros([X.shape[0],1])
    
    for i, x in enumerate(X):
        if x[index] == 1:
            np_vector[i] = np.sign(weight)
    
    return np_vector

In [24]:
# I need to generate the ht's

ht_1_input = np_present_by_doc(data, 1, word_weight[1])
ht_2_input = np_present_by_doc(data, 2, word_weight[2])
ht_3_input = np_present_by_doc(data, 3, word_weight[3])
ht_4_input = np_present_by_doc(data, 4, word_weight[4])

In [25]:
ht_1_input.shape

(16, 1)

In [26]:
ht_input = np.hstack([ht_1_input, ht_2_input, ht_3_input, ht_4_input])

In [27]:
ht_input

array([[ 0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -1.],
       [-1.,  1.,  0.,  0.],
       [-1.,  0.,  1.,  0.],
       [-1.,  0.,  0., -1.],
       [ 0.,  1.,  1.,  0.],
       [ 0.,  1.,  0., -1.],
       [ 0.,  0.,  1., -1.],
       [-1.,  1.,  1.,  0.],
       [-1.,  1.,  0., -1.],
       [-1.,  0.,  1., -1.],
       [ 0.,  1.,  1., -1.],
       [-1.,  1.,  1., -1.]])

In [28]:
for i in range(len(ht_1_input)):
    print(data[i], ht_4_input[i])

[1. 0. 0. 0. 0.] [0.]
[1. 1. 0. 0. 0.] [0.]
[1. 0. 1. 0. 0.] [0.]
[1. 0. 0. 1. 0.] [0.]
[1. 0. 0. 0. 1.] [-1.]
[1. 1. 1. 0. 0.] [0.]
[1. 1. 0. 1. 0.] [0.]
[1. 1. 0. 0. 1.] [-1.]
[1. 0. 1. 1. 0.] [0.]
[1. 0. 1. 0. 1.] [-1.]
[1. 0. 0. 1. 1.] [-1.]
[1. 1. 1. 1. 0.] [0.]
[1. 1. 1. 0. 1.] [-1.]
[1. 1. 0. 1. 1.] [-1.]
[1. 0. 1. 1. 1.] [-1.]
[1. 1. 1. 1. 1.] [-1.]


In [ ]:
# # adjust y to -> [-1, 1]
# label_tanh = []
# for i in label:
#     if i==0:
#         label_tanh.append(-1)
#     else:
#         label_tanh.append(1)
# label_tanh = np.asarray(label_tanh)

In [29]:
base_model.fit(data[1:15], label[1:15], epochs=1000, batch_size=1, verbose=2)

Epoch 1/1000
 - 0s - loss: 1.6720 - acc: 0.2857
Epoch 2/1000
 - 0s - loss: 1.6626 - acc: 0.2857
Epoch 3/1000
 - 0s - loss: 1.6542 - acc: 0.2857
Epoch 4/1000
 - 0s - loss: 1.6506 - acc: 0.2857
Epoch 5/1000
 - 0s - loss: 1.6373 - acc: 0.2857
Epoch 6/1000
 - 0s - loss: 1.6292 - acc: 0.2857
Epoch 7/1000
 - 0s - loss: 1.6209 - acc: 0.2857
Epoch 8/1000
 - 0s - loss: 1.6115 - acc: 0.2143
Epoch 9/1000
 - 0s - loss: 1.6051 - acc: 0.2143
Epoch 10/1000
 - 0s - loss: 1.5915 - acc: 0.2143
Epoch 11/1000
 - 0s - loss: 1.5823 - acc: 0.2143
Epoch 12/1000
 - 0s - loss: 1.5725 - acc: 0.2143
Epoch 13/1000
 - 0s - loss: 1.5630 - acc: 0.2143
Epoch 14/1000
 - 0s - loss: 1.5550 - acc: 0.2143
Epoch 15/1000
 - 0s - loss: 1.5402 - acc: 0.2143
Epoch 16/1000
 - 0s - loss: 1.5340 - acc: 0.2143
Epoch 17/1000
 - 0s - loss: 1.5209 - acc: 0.2143
Epoch 18/1000
 - 0s - loss: 1.5072 - acc: 0.2143
Epoch 19/1000
 - 0s - loss: 1.4968 - acc: 0.2143
Epoch 20/1000
 - 0s - loss: 1.4855 - acc: 0.2143
Epoch 21/1000
 - 0s - loss: 1

 - 0s - loss: 0.4524 - acc: 0.4286
Epoch 168/1000
 - 0s - loss: 0.4489 - acc: 0.4286
Epoch 169/1000
 - 0s - loss: 0.4465 - acc: 0.4286
Epoch 170/1000
 - 0s - loss: 0.4436 - acc: 0.4286
Epoch 171/1000
 - 0s - loss: 0.4408 - acc: 0.4286
Epoch 172/1000
 - 0s - loss: 0.4390 - acc: 0.4286
Epoch 173/1000
 - 0s - loss: 0.4362 - acc: 0.4286
Epoch 174/1000
 - 0s - loss: 0.4325 - acc: 0.4286
Epoch 175/1000
 - 0s - loss: 0.4303 - acc: 0.4286
Epoch 176/1000
 - 0s - loss: 0.4276 - acc: 0.4286
Epoch 177/1000
 - 0s - loss: 0.4249 - acc: 0.4286
Epoch 178/1000
 - 0s - loss: 0.4223 - acc: 0.4286
Epoch 179/1000
 - 0s - loss: 0.4202 - acc: 0.4286
Epoch 180/1000
 - 0s - loss: 0.4177 - acc: 0.4286
Epoch 181/1000
 - 0s - loss: 0.4151 - acc: 0.4286
Epoch 182/1000
 - 0s - loss: 0.4130 - acc: 0.4286
Epoch 183/1000
 - 0s - loss: 0.4101 - acc: 0.4286
Epoch 184/1000
 - 0s - loss: 0.4076 - acc: 0.4286
Epoch 185/1000
 - 0s - loss: 0.4054 - acc: 0.4286
Epoch 186/1000
 - 0s - loss: 0.4031 - acc: 0.4286
Epoch 187/1000


 - 0s - loss: 0.2123 - acc: 0.5714
Epoch 332/1000
 - 0s - loss: 0.2118 - acc: 0.5714
Epoch 333/1000
 - 0s - loss: 0.2112 - acc: 0.5714
Epoch 334/1000
 - 0s - loss: 0.2103 - acc: 0.5714
Epoch 335/1000
 - 0s - loss: 0.2095 - acc: 0.5714
Epoch 336/1000
 - 0s - loss: 0.2087 - acc: 0.5714
Epoch 337/1000
 - 0s - loss: 0.2084 - acc: 0.5714
Epoch 338/1000
 - 0s - loss: 0.2072 - acc: 0.5714
Epoch 339/1000
 - 0s - loss: 0.2064 - acc: 0.5714
Epoch 340/1000
 - 0s - loss: 0.2062 - acc: 0.5714
Epoch 341/1000
 - 0s - loss: 0.2052 - acc: 0.5714
Epoch 342/1000
 - 0s - loss: 0.2046 - acc: 0.5714
Epoch 343/1000
 - 0s - loss: 0.2036 - acc: 0.5714
Epoch 344/1000
 - 0s - loss: 0.2029 - acc: 0.5714
Epoch 345/1000
 - 0s - loss: 0.2024 - acc: 0.5714
Epoch 346/1000
 - 0s - loss: 0.2016 - acc: 0.5714
Epoch 347/1000
 - 0s - loss: 0.2012 - acc: 0.5714
Epoch 348/1000
 - 0s - loss: 0.2000 - acc: 0.5714
Epoch 349/1000
 - 0s - loss: 0.1995 - acc: 0.5714
Epoch 350/1000
 - 0s - loss: 0.1993 - acc: 0.5714
Epoch 351/1000


Epoch 496/1000
 - 0s - loss: 0.1235 - acc: 0.8571
Epoch 497/1000
 - 0s - loss: 0.1232 - acc: 0.8571
Epoch 498/1000
 - 0s - loss: 0.1228 - acc: 0.8571
Epoch 499/1000
 - 0s - loss: 0.1225 - acc: 0.8571
Epoch 500/1000
 - 0s - loss: 0.1220 - acc: 0.8571
Epoch 501/1000
 - 0s - loss: 0.1216 - acc: 0.8571
Epoch 502/1000
 - 0s - loss: 0.1211 - acc: 0.8571
Epoch 503/1000
 - 0s - loss: 0.1208 - acc: 0.8571
Epoch 504/1000
 - 0s - loss: 0.1205 - acc: 0.8571
Epoch 505/1000
 - 0s - loss: 0.1202 - acc: 0.8571
Epoch 506/1000
 - 0s - loss: 0.1197 - acc: 0.8571
Epoch 507/1000
 - 0s - loss: 0.1194 - acc: 0.8571
Epoch 508/1000
 - 0s - loss: 0.1193 - acc: 0.8571
Epoch 509/1000
 - 0s - loss: 0.1188 - acc: 0.8571
Epoch 510/1000
 - 0s - loss: 0.1182 - acc: 0.8571
Epoch 511/1000
 - 0s - loss: 0.1178 - acc: 0.8571
Epoch 512/1000
 - 0s - loss: 0.1175 - acc: 0.8571
Epoch 513/1000
 - 0s - loss: 0.1171 - acc: 0.8571
Epoch 514/1000
 - 0s - loss: 0.1169 - acc: 0.8571
Epoch 515/1000
 - 0s - loss: 0.1167 - acc: 0.8571


Epoch 660/1000
 - 0s - loss: 0.0743 - acc: 1.0000
Epoch 661/1000
 - 0s - loss: 0.0745 - acc: 1.0000
Epoch 662/1000
 - 0s - loss: 0.0738 - acc: 1.0000
Epoch 663/1000
 - 0s - loss: 0.0737 - acc: 1.0000
Epoch 664/1000
 - 0s - loss: 0.0736 - acc: 1.0000
Epoch 665/1000
 - 0s - loss: 0.0735 - acc: 1.0000
Epoch 666/1000
 - 0s - loss: 0.0728 - acc: 1.0000
Epoch 667/1000
 - 0s - loss: 0.0729 - acc: 1.0000
Epoch 668/1000
 - 0s - loss: 0.0726 - acc: 1.0000
Epoch 669/1000
 - 0s - loss: 0.0722 - acc: 1.0000
Epoch 670/1000
 - 0s - loss: 0.0720 - acc: 1.0000
Epoch 671/1000
 - 0s - loss: 0.0718 - acc: 1.0000
Epoch 672/1000
 - 0s - loss: 0.0718 - acc: 1.0000
Epoch 673/1000
 - 0s - loss: 0.0715 - acc: 1.0000
Epoch 674/1000
 - 0s - loss: 0.0712 - acc: 1.0000
Epoch 675/1000
 - 0s - loss: 0.0709 - acc: 1.0000
Epoch 676/1000
 - 0s - loss: 0.0708 - acc: 1.0000
Epoch 677/1000
 - 0s - loss: 0.0707 - acc: 1.0000
Epoch 678/1000
 - 0s - loss: 0.0701 - acc: 1.0000
Epoch 679/1000
 - 0s - loss: 0.0700 - acc: 1.0000


Epoch 824/1000
 - 0s - loss: 0.0447 - acc: 1.0000
Epoch 825/1000
 - 0s - loss: 0.0445 - acc: 1.0000
Epoch 826/1000
 - 0s - loss: 0.0445 - acc: 1.0000
Epoch 827/1000
 - 0s - loss: 0.0443 - acc: 1.0000
Epoch 828/1000
 - 0s - loss: 0.0442 - acc: 1.0000
Epoch 829/1000
 - 0s - loss: 0.0440 - acc: 1.0000
Epoch 830/1000
 - 0s - loss: 0.0440 - acc: 1.0000
Epoch 831/1000
 - 0s - loss: 0.0438 - acc: 1.0000
Epoch 832/1000
 - 0s - loss: 0.0436 - acc: 1.0000
Epoch 833/1000
 - 0s - loss: 0.0435 - acc: 1.0000
Epoch 834/1000
 - 0s - loss: 0.0433 - acc: 1.0000
Epoch 835/1000
 - 0s - loss: 0.0432 - acc: 1.0000
Epoch 836/1000
 - 0s - loss: 0.0431 - acc: 1.0000
Epoch 837/1000
 - 0s - loss: 0.0430 - acc: 1.0000
Epoch 838/1000
 - 0s - loss: 0.0429 - acc: 1.0000
Epoch 839/1000
 - 0s - loss: 0.0427 - acc: 1.0000
Epoch 840/1000
 - 0s - loss: 0.0425 - acc: 1.0000
Epoch 841/1000
 - 0s - loss: 0.0424 - acc: 1.0000
Epoch 842/1000
 - 0s - loss: 0.0423 - acc: 1.0000
Epoch 843/1000
 - 0s - loss: 0.0422 - acc: 1.0000


Epoch 988/1000
 - 0s - loss: 0.0269 - acc: 1.0000
Epoch 989/1000
 - 0s - loss: 0.0268 - acc: 1.0000
Epoch 990/1000
 - 0s - loss: 0.0267 - acc: 1.0000
Epoch 991/1000
 - 0s - loss: 0.0266 - acc: 1.0000
Epoch 992/1000
 - 0s - loss: 0.0265 - acc: 1.0000
Epoch 993/1000
 - 0s - loss: 0.0264 - acc: 1.0000
Epoch 994/1000
 - 0s - loss: 0.0263 - acc: 1.0000
Epoch 995/1000
 - 0s - loss: 0.0262 - acc: 1.0000
Epoch 996/1000
 - 0s - loss: 0.0262 - acc: 1.0000
Epoch 997/1000
 - 0s - loss: 0.0261 - acc: 1.0000
Epoch 998/1000
 - 0s - loss: 0.0260 - acc: 1.0000
Epoch 999/1000
 - 0s - loss: 0.0259 - acc: 1.0000
Epoch 1000/1000
 - 0s - loss: 0.0258 - acc: 1.0000


In [30]:
y = base_model.predict(data[1:15])
for i in range(len(y)):
    print(data[i+1], label[i+1], y[i])

[1. 1. 0. 0. 0.] -1.0 [-0.73418593]
[1. 0. 1. 0. 0.] 1.0 [0.90499914]
[1. 0. 0. 1. 0.] 1.0 [0.9051766]
[1. 0. 0. 0. 1.] -1.0 [-0.9955049]
[1. 1. 1. 0. 0.] 1.0 [0.78906035]
[1. 1. 0. 1. 0.] 1.0 [0.7894305]
[1. 1. 0. 0. 1.] -1.0 [-0.9980963]
[1. 0. 1. 1. 0.] 1.0 [0.9982026]
[1. 0. 1. 0. 1.] -1.0 [-0.77834153]
[1. 0. 0. 1. 1.] -1.0 [-0.77795434]
[1. 1. 1. 1. 0.] 1.0 [0.99575573]
[1. 1. 1. 0. 1.] -1.0 [-0.89984185]
[1. 1. 0. 1. 1.] -1.0 [-0.8996549]
[1. 0. 1. 1. 1.] 1.0 [0.74716973]


In [31]:
combined_model.fit([data[1:15], ht_1_input[1:15], ht_2_input[1:15], ht_3_input[1:15], ht_4_input[1:15]], 
                   np_label[1:15], 
                   epochs=500,
                   batch_size=1,
                   verbose=2)

Epoch 1/500
 - 1s - loss: 0.4670 - acc: 0.0714
Epoch 2/500
 - 0s - loss: 0.4613 - acc: 0.0714
Epoch 3/500
 - 0s - loss: 0.4559 - acc: 0.0714
Epoch 4/500
 - 0s - loss: 0.4505 - acc: 0.0714
Epoch 5/500
 - 0s - loss: 0.4450 - acc: 0.0714
Epoch 6/500
 - 0s - loss: 0.4394 - acc: 0.0714
Epoch 7/500
 - 0s - loss: 0.4342 - acc: 0.0714
Epoch 8/500
 - 0s - loss: 0.4289 - acc: 0.0714
Epoch 9/500
 - 0s - loss: 0.4236 - acc: 0.0714
Epoch 10/500
 - 0s - loss: 0.4179 - acc: 0.0714
Epoch 11/500
 - 0s - loss: 0.4127 - acc: 0.0714
Epoch 12/500
 - 0s - loss: 0.4075 - acc: 0.0714
Epoch 13/500
 - 0s - loss: 0.4023 - acc: 0.0714
Epoch 14/500
 - 0s - loss: 0.3972 - acc: 0.0714
Epoch 15/500
 - 0s - loss: 0.3918 - acc: 0.0714
Epoch 16/500
 - 0s - loss: 0.3868 - acc: 0.0714
Epoch 17/500
 - 0s - loss: 0.3817 - acc: 0.0714
Epoch 18/500
 - 0s - loss: 0.3769 - acc: 0.0714
Epoch 19/500
 - 0s - loss: 0.3716 - acc: 0.2857
Epoch 20/500
 - 0s - loss: 0.3668 - acc: 0.2857
Epoch 21/500
 - 0s - loss: 0.3620 - acc: 0.2857
E

Epoch 171/500
 - 0s - loss: 0.1577 - acc: 0.7857
Epoch 172/500
 - 0s - loss: 0.1572 - acc: 0.7857
Epoch 173/500
 - 0s - loss: 0.1567 - acc: 0.7857
Epoch 174/500
 - 0s - loss: 0.1562 - acc: 0.7857
Epoch 175/500
 - 0s - loss: 0.1558 - acc: 0.7857
Epoch 176/500
 - 0s - loss: 0.1553 - acc: 0.7857
Epoch 177/500
 - 0s - loss: 0.1549 - acc: 0.7857
Epoch 178/500
 - 0s - loss: 0.1544 - acc: 0.7857
Epoch 179/500
 - 0s - loss: 0.1539 - acc: 0.7857
Epoch 180/500
 - 0s - loss: 0.1535 - acc: 0.7857
Epoch 181/500
 - 0s - loss: 0.1530 - acc: 0.7857
Epoch 182/500
 - 0s - loss: 0.1524 - acc: 0.7857
Epoch 183/500
 - 0s - loss: 0.1521 - acc: 0.7857
Epoch 184/500
 - 0s - loss: 0.1515 - acc: 0.7857
Epoch 185/500
 - 0s - loss: 0.1510 - acc: 0.7857
Epoch 186/500
 - 0s - loss: 0.1505 - acc: 0.7857
Epoch 187/500
 - 0s - loss: 0.1501 - acc: 0.7857
Epoch 188/500
 - 0s - loss: 0.1496 - acc: 0.7857
Epoch 189/500
 - 0s - loss: 0.1491 - acc: 0.7857
Epoch 190/500
 - 0s - loss: 0.1486 - acc: 0.7857
Epoch 191/500
 - 0s 

 - 0s - loss: 0.0588 - acc: 1.0000
Epoch 339/500
 - 0s - loss: 0.0583 - acc: 1.0000
Epoch 340/500
 - 0s - loss: 0.0577 - acc: 1.0000
Epoch 341/500
 - 0s - loss: 0.0572 - acc: 1.0000
Epoch 342/500
 - 0s - loss: 0.0566 - acc: 1.0000
Epoch 343/500
 - 0s - loss: 0.0560 - acc: 1.0000
Epoch 344/500
 - 0s - loss: 0.0556 - acc: 1.0000
Epoch 345/500
 - 0s - loss: 0.0549 - acc: 1.0000
Epoch 346/500
 - 0s - loss: 0.0544 - acc: 1.0000
Epoch 347/500
 - 0s - loss: 0.0539 - acc: 1.0000
Epoch 348/500
 - 0s - loss: 0.0534 - acc: 1.0000
Epoch 349/500
 - 0s - loss: 0.0530 - acc: 1.0000
Epoch 350/500
 - 0s - loss: 0.0524 - acc: 1.0000
Epoch 351/500
 - 0s - loss: 0.0519 - acc: 1.0000
Epoch 352/500
 - 0s - loss: 0.0515 - acc: 1.0000
Epoch 353/500
 - 0s - loss: 0.0509 - acc: 1.0000
Epoch 354/500
 - 0s - loss: 0.0504 - acc: 1.0000
Epoch 355/500
 - 0s - loss: 0.0500 - acc: 1.0000
Epoch 356/500
 - 0s - loss: 0.0497 - acc: 1.0000
Epoch 357/500
 - 0s - loss: 0.0491 - acc: 1.0000
Epoch 358/500
 - 0s - loss: 0.0486

In [32]:
net_weight = combined_model.get_weights()
net_weight

[array([[-0.22956319],
        [-0.4299929 ],
        [ 2.0729353 ],
        [ 2.3534873 ],
        [-2.8235989 ]], dtype=float32),
 array([-0.70369315], dtype=float32),
 array([[2.435016]], dtype=float32),
 array([[-0.42480403]], dtype=float32),
 array([[2.3615427]], dtype=float32),
 array([[1.7641373]], dtype=float32),
 array([[-1.7631199 ],
        [-0.57229006],
        [ 1.5718032 ],
        [-1.6381247 ]], dtype=float32),
 array([0.60496473], dtype=float32)]

In [33]:
def report():
    for i in range(1,15):
        bm = base_model.predict(np.reshape(data[i], (1,5)))
        
        cm = combined_model.predict([np.reshape(data[i], (1,5)), 
                                np.reshape(ht_input[i], (1,4))])
        
#         document_output = 'multiply'
#         document_predict = Model(inputs=combined_model.input,
#                                      outputs=combined_model.get_layer(document_output).output)
#         doc_output = document_predict.predict([np.reshape(data[i], (1,5)), 
#                                       ht_1_input[i], 
#                                       ht_2_input[i], 
#                                       ht_3_input[i], 
#                                       ht_4_input[i]])
        
        layer_name = 'concatenate'
        concat_after_relu = Model(inputs=combined_model.input,
                                     outputs=combined_model.get_layer(layer_name).output)
        concat_output = concat_after_relu.predict([np.reshape(data[i], (1,5)), 
                                     np.reshape(ht_input[i], (1,4))])
        
        print(data[i], '\t', np_label[i], '\t', bm.flatten(), '\t', cm.flatten(), '\t',concat_output.flatten())
    

In [34]:
ht_input.shape

(16, 4)

In [35]:
# report()
# need to modify the ht_input

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 5 array(s), but instead got the following list of 2 arrays: [array([[1., 1., 0., 0., 0.]]), array([[-1.,  0.,  0.,  0.]])]...

In [ ]:
word_weight

In [ ]:
from keras.models import load_model

# base_model.save('overfit_base_model.h5')
# combined_model.save('overfit_combined_model.h5')

In [ ]:
# bm = base_model.predict(np.reshape(data[test_idx], (1,5)))
# print(bm)

# before jointly trained.
# 0.7995629

### IMDB network Model
#### concat and split layer test

In [36]:
# build the combined model
# Combined model
human_terms_len = 4

base_model = build_base_model(data.shape[1])

combined_input_layer = Input(shape=(data.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# multiply and pass it into relu
# initialize relu layer

relu_layer = Dense(1, activation='relu', name='relu_layer', use_bias=False, kernel_initializer='ones')

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(relu_layer(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 5)                 0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 4)            0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 5)            0                                            
_________________________________________________

In [37]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [38]:
base_model.fit(data[1:15], label[1:15], epochs=1000, batch_size=1, verbose=2)

Epoch 1/1000
 - 0s - loss: 0.9015 - acc: 0.2857
Epoch 2/1000
 - 0s - loss: 0.8843 - acc: 0.2857
Epoch 3/1000
 - 0s - loss: 0.8724 - acc: 0.2857
Epoch 4/1000
 - 0s - loss: 0.8577 - acc: 0.2857
Epoch 5/1000
 - 0s - loss: 0.8435 - acc: 0.2857
Epoch 6/1000
 - 0s - loss: 0.8345 - acc: 0.2857
Epoch 7/1000
 - 0s - loss: 0.8206 - acc: 0.3571
Epoch 8/1000
 - 0s - loss: 0.8093 - acc: 0.3571
Epoch 9/1000
 - 0s - loss: 0.7982 - acc: 0.3571
Epoch 10/1000
 - 0s - loss: 0.7874 - acc: 0.3571
Epoch 11/1000
 - 0s - loss: 0.7782 - acc: 0.3571
Epoch 12/1000
 - 0s - loss: 0.7710 - acc: 0.2857
Epoch 13/1000
 - 0s - loss: 0.7588 - acc: 0.2857
Epoch 14/1000
 - 0s - loss: 0.7487 - acc: 0.2857
Epoch 15/1000
 - 0s - loss: 0.7415 - acc: 0.2857
Epoch 16/1000
 - 0s - loss: 0.7330 - acc: 0.2857
Epoch 17/1000
 - 0s - loss: 0.7243 - acc: 0.2857
Epoch 18/1000
 - 0s - loss: 0.7209 - acc: 0.2857
Epoch 19/1000
 - 0s - loss: 0.7093 - acc: 0.2857
Epoch 20/1000
 - 0s - loss: 0.7018 - acc: 0.2857
Epoch 21/1000
 - 0s - loss: 0

 - 0s - loss: 0.2747 - acc: 0.2857
Epoch 168/1000
 - 0s - loss: 0.2733 - acc: 0.2857
Epoch 169/1000
 - 0s - loss: 0.2714 - acc: 0.2857
Epoch 170/1000
 - 0s - loss: 0.2703 - acc: 0.2857
Epoch 171/1000
 - 0s - loss: 0.2691 - acc: 0.2857
Epoch 172/1000
 - 0s - loss: 0.2680 - acc: 0.2857
Epoch 173/1000
 - 0s - loss: 0.2661 - acc: 0.2857
Epoch 174/1000
 - 0s - loss: 0.2649 - acc: 0.2857
Epoch 175/1000
 - 0s - loss: 0.2636 - acc: 0.2857
Epoch 176/1000
 - 0s - loss: 0.2620 - acc: 0.2857
Epoch 177/1000
 - 0s - loss: 0.2608 - acc: 0.2857
Epoch 178/1000
 - 0s - loss: 0.2594 - acc: 0.2857
Epoch 179/1000
 - 0s - loss: 0.2579 - acc: 0.2857
Epoch 180/1000
 - 0s - loss: 0.2571 - acc: 0.2857
Epoch 181/1000
 - 0s - loss: 0.2555 - acc: 0.2857
Epoch 182/1000
 - 0s - loss: 0.2543 - acc: 0.2857
Epoch 183/1000
 - 0s - loss: 0.2529 - acc: 0.2857
Epoch 184/1000
 - 0s - loss: 0.2519 - acc: 0.2857
Epoch 185/1000
 - 0s - loss: 0.2504 - acc: 0.2857
Epoch 186/1000
 - 0s - loss: 0.2495 - acc: 0.2857
Epoch 187/1000


Epoch 332/1000
 - 0s - loss: 0.1336 - acc: 0.8571
Epoch 333/1000
 - 0s - loss: 0.1328 - acc: 0.8571
Epoch 334/1000
 - 0s - loss: 0.1324 - acc: 0.8571
Epoch 335/1000
 - 0s - loss: 0.1319 - acc: 0.8571
Epoch 336/1000
 - 0s - loss: 0.1315 - acc: 0.8571
Epoch 337/1000
 - 0s - loss: 0.1308 - acc: 0.8571
Epoch 338/1000
 - 0s - loss: 0.1307 - acc: 0.8571
Epoch 339/1000
 - 0s - loss: 0.1297 - acc: 0.8571
Epoch 340/1000
 - 0s - loss: 0.1296 - acc: 0.8571
Epoch 341/1000
 - 0s - loss: 0.1288 - acc: 0.8571
Epoch 342/1000
 - 0s - loss: 0.1285 - acc: 0.8571
Epoch 343/1000
 - 0s - loss: 0.1279 - acc: 0.8571
Epoch 344/1000
 - 0s - loss: 0.1273 - acc: 0.8571
Epoch 345/1000
 - 0s - loss: 0.1269 - acc: 0.9286
Epoch 346/1000
 - 0s - loss: 0.1265 - acc: 0.8571
Epoch 347/1000
 - 0s - loss: 0.1259 - acc: 1.0000
Epoch 348/1000
 - 0s - loss: 0.1257 - acc: 0.9286
Epoch 349/1000
 - 0s - loss: 0.1249 - acc: 0.9286
Epoch 350/1000
 - 0s - loss: 0.1248 - acc: 0.9286
Epoch 351/1000
 - 0s - loss: 0.1241 - acc: 0.9286


Epoch 496/1000
 - 0s - loss: 0.0746 - acc: 1.0000
Epoch 497/1000
 - 0s - loss: 0.0741 - acc: 1.0000
Epoch 498/1000
 - 0s - loss: 0.0740 - acc: 1.0000
Epoch 499/1000
 - 0s - loss: 0.0736 - acc: 1.0000
Epoch 500/1000
 - 0s - loss: 0.0736 - acc: 1.0000
Epoch 501/1000
 - 0s - loss: 0.0731 - acc: 1.0000
Epoch 502/1000
 - 0s - loss: 0.0729 - acc: 1.0000
Epoch 503/1000
 - 0s - loss: 0.0726 - acc: 1.0000
Epoch 504/1000
 - 0s - loss: 0.0724 - acc: 1.0000
Epoch 505/1000
 - 0s - loss: 0.0721 - acc: 1.0000
Epoch 506/1000
 - 0s - loss: 0.0719 - acc: 1.0000
Epoch 507/1000
 - 0s - loss: 0.0716 - acc: 1.0000
Epoch 508/1000
 - 0s - loss: 0.0714 - acc: 1.0000
Epoch 509/1000
 - 0s - loss: 0.0712 - acc: 1.0000
Epoch 510/1000
 - 0s - loss: 0.0712 - acc: 1.0000
Epoch 511/1000
 - 0s - loss: 0.0707 - acc: 1.0000
Epoch 512/1000
 - 0s - loss: 0.0704 - acc: 1.0000
Epoch 513/1000
 - 0s - loss: 0.0702 - acc: 1.0000
Epoch 514/1000
 - 0s - loss: 0.0700 - acc: 1.0000
Epoch 515/1000
 - 0s - loss: 0.0698 - acc: 1.0000


Epoch 660/1000
 - 0s - loss: 0.0431 - acc: 1.0000
Epoch 661/1000
 - 0s - loss: 0.0430 - acc: 1.0000
Epoch 662/1000
 - 0s - loss: 0.0429 - acc: 1.0000
Epoch 663/1000
 - 0s - loss: 0.0427 - acc: 1.0000
Epoch 664/1000
 - 0s - loss: 0.0425 - acc: 1.0000
Epoch 665/1000
 - 0s - loss: 0.0425 - acc: 1.0000
Epoch 666/1000
 - 0s - loss: 0.0422 - acc: 1.0000
Epoch 667/1000
 - 0s - loss: 0.0422 - acc: 1.0000
Epoch 668/1000
 - 0s - loss: 0.0420 - acc: 1.0000
Epoch 669/1000
 - 0s - loss: 0.0419 - acc: 1.0000
Epoch 670/1000
 - 0s - loss: 0.0417 - acc: 1.0000
Epoch 671/1000
 - 0s - loss: 0.0416 - acc: 1.0000
Epoch 672/1000
 - 0s - loss: 0.0414 - acc: 1.0000
Epoch 673/1000
 - 0s - loss: 0.0413 - acc: 1.0000
Epoch 674/1000
 - 0s - loss: 0.0412 - acc: 1.0000
Epoch 675/1000
 - 0s - loss: 0.0411 - acc: 1.0000
Epoch 676/1000
 - 0s - loss: 0.0410 - acc: 1.0000
Epoch 677/1000
 - 0s - loss: 0.0408 - acc: 1.0000
Epoch 678/1000
 - 0s - loss: 0.0408 - acc: 1.0000
Epoch 679/1000
 - 0s - loss: 0.0405 - acc: 1.0000


Epoch 824/1000
 - 0s - loss: 0.0253 - acc: 1.0000
Epoch 825/1000
 - 0s - loss: 0.0252 - acc: 1.0000
Epoch 826/1000
 - 0s - loss: 0.0251 - acc: 1.0000
Epoch 827/1000
 - 0s - loss: 0.0250 - acc: 1.0000
Epoch 828/1000
 - 0s - loss: 0.0250 - acc: 1.0000
Epoch 829/1000
 - 0s - loss: 0.0249 - acc: 1.0000
Epoch 830/1000
 - 0s - loss: 0.0248 - acc: 1.0000
Epoch 831/1000
 - 0s - loss: 0.0248 - acc: 1.0000
Epoch 832/1000
 - 0s - loss: 0.0248 - acc: 1.0000
Epoch 833/1000
 - 0s - loss: 0.0246 - acc: 1.0000
Epoch 834/1000
 - 0s - loss: 0.0245 - acc: 1.0000
Epoch 835/1000
 - 0s - loss: 0.0244 - acc: 1.0000
Epoch 836/1000
 - 0s - loss: 0.0243 - acc: 1.0000
Epoch 837/1000
 - 0s - loss: 0.0243 - acc: 1.0000
Epoch 838/1000
 - 0s - loss: 0.0242 - acc: 1.0000
Epoch 839/1000
 - 0s - loss: 0.0241 - acc: 1.0000
Epoch 840/1000
 - 0s - loss: 0.0241 - acc: 1.0000
Epoch 841/1000
 - 0s - loss: 0.0240 - acc: 1.0000
Epoch 842/1000
 - 0s - loss: 0.0239 - acc: 1.0000
Epoch 843/1000
 - 0s - loss: 0.0238 - acc: 1.0000


Epoch 988/1000
 - 0s - loss: 0.0149 - acc: 1.0000
Epoch 989/1000
 - 0s - loss: 0.0148 - acc: 1.0000
Epoch 990/1000
 - 0s - loss: 0.0148 - acc: 1.0000
Epoch 991/1000
 - 0s - loss: 0.0148 - acc: 1.0000
Epoch 992/1000
 - 0s - loss: 0.0147 - acc: 1.0000
Epoch 993/1000
 - 0s - loss: 0.0147 - acc: 1.0000
Epoch 994/1000
 - 0s - loss: 0.0146 - acc: 1.0000
Epoch 995/1000
 - 0s - loss: 0.0146 - acc: 1.0000
Epoch 996/1000
 - 0s - loss: 0.0145 - acc: 1.0000
Epoch 997/1000
 - 0s - loss: 0.0145 - acc: 1.0000
Epoch 998/1000
 - 0s - loss: 0.0145 - acc: 1.0000
Epoch 999/1000
 - 0s - loss: 0.0144 - acc: 1.0000
Epoch 1000/1000
 - 0s - loss: 0.0143 - acc: 1.0000


In [39]:
combined_model.fit([data[1:15], ht_input[1:15]], 
                   np_label[1:15], 
                   epochs=500,
                   batch_size=1,
                   verbose=2)

Epoch 1/500
 - 1s - loss: 0.4586 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 0.4527 - acc: 0.0000e+00
Epoch 3/500
 - 0s - loss: 0.4473 - acc: 0.0000e+00
Epoch 4/500
 - 0s - loss: 0.4413 - acc: 0.0000e+00
Epoch 5/500
 - 0s - loss: 0.4361 - acc: 0.0000e+00
Epoch 6/500
 - 0s - loss: 0.4307 - acc: 0.0000e+00
Epoch 7/500
 - 0s - loss: 0.4250 - acc: 0.0000e+00
Epoch 8/500
 - 0s - loss: 0.4200 - acc: 0.0000e+00
Epoch 9/500
 - 0s - loss: 0.4147 - acc: 0.0000e+00
Epoch 10/500
 - 0s - loss: 0.4098 - acc: 0.0000e+00
Epoch 11/500
 - 0s - loss: 0.4051 - acc: 0.0000e+00
Epoch 12/500
 - 0s - loss: 0.3998 - acc: 0.0000e+00
Epoch 13/500
 - 0s - loss: 0.3952 - acc: 0.0000e+00
Epoch 14/500
 - 0s - loss: 0.3907 - acc: 0.0000e+00
Epoch 15/500
 - 0s - loss: 0.3863 - acc: 0.0000e+00
Epoch 16/500
 - 0s - loss: 0.3815 - acc: 0.0000e+00
Epoch 17/500
 - 0s - loss: 0.3777 - acc: 0.0000e+00
Epoch 18/500
 - 0s - loss: 0.3731 - acc: 0.0000e+00
Epoch 19/500
 - 0s - loss: 0.3690 - acc: 0.0000e+00
Epoch 20/500
 - 0s - 

Epoch 169/500
 - 0s - loss: 0.1264 - acc: 1.0000
Epoch 170/500
 - 0s - loss: 0.1235 - acc: 1.0000
Epoch 171/500
 - 0s - loss: 0.1206 - acc: 1.0000
Epoch 172/500
 - 0s - loss: 0.1178 - acc: 1.0000
Epoch 173/500
 - 0s - loss: 0.1151 - acc: 1.0000
Epoch 174/500
 - 0s - loss: 0.1123 - acc: 1.0000
Epoch 175/500
 - 0s - loss: 0.1096 - acc: 1.0000
Epoch 176/500
 - 0s - loss: 0.1070 - acc: 1.0000
Epoch 177/500
 - 0s - loss: 0.1044 - acc: 1.0000
Epoch 178/500
 - 0s - loss: 0.1019 - acc: 1.0000
Epoch 179/500
 - 0s - loss: 0.0994 - acc: 1.0000
Epoch 180/500
 - 0s - loss: 0.0970 - acc: 1.0000
Epoch 181/500
 - 0s - loss: 0.0946 - acc: 1.0000
Epoch 182/500
 - 0s - loss: 0.0924 - acc: 1.0000
Epoch 183/500
 - 0s - loss: 0.0901 - acc: 1.0000
Epoch 184/500
 - 0s - loss: 0.0879 - acc: 1.0000
Epoch 185/500
 - 0s - loss: 0.0858 - acc: 1.0000
Epoch 186/500
 - 0s - loss: 0.0837 - acc: 1.0000
Epoch 187/500
 - 0s - loss: 0.0817 - acc: 1.0000
Epoch 188/500
 - 0s - loss: 0.0798 - acc: 1.0000
Epoch 189/500
 - 0s 

 - 0s - loss: 0.0074 - acc: 1.0000
Epoch 337/500
 - 0s - loss: 0.0072 - acc: 1.0000
Epoch 338/500
 - 0s - loss: 0.0072 - acc: 1.0000
Epoch 339/500
 - 0s - loss: 0.0071 - acc: 1.0000
Epoch 340/500
 - 0s - loss: 0.0070 - acc: 1.0000
Epoch 341/500
 - 0s - loss: 0.0069 - acc: 1.0000
Epoch 342/500
 - 0s - loss: 0.0068 - acc: 1.0000
Epoch 343/500
 - 0s - loss: 0.0067 - acc: 1.0000
Epoch 344/500
 - 0s - loss: 0.0066 - acc: 1.0000
Epoch 345/500
 - 0s - loss: 0.0065 - acc: 1.0000
Epoch 346/500
 - 0s - loss: 0.0064 - acc: 1.0000
Epoch 347/500
 - 0s - loss: 0.0064 - acc: 1.0000
Epoch 348/500
 - 0s - loss: 0.0063 - acc: 1.0000
Epoch 349/500
 - 0s - loss: 0.0062 - acc: 1.0000
Epoch 350/500
 - 0s - loss: 0.0061 - acc: 1.0000
Epoch 351/500
 - 0s - loss: 0.0061 - acc: 1.0000
Epoch 352/500
 - 0s - loss: 0.0060 - acc: 1.0000
Epoch 353/500
 - 0s - loss: 0.0059 - acc: 1.0000
Epoch 354/500
 - 0s - loss: 0.0058 - acc: 1.0000
Epoch 355/500
 - 0s - loss: 0.0058 - acc: 1.0000
Epoch 356/500
 - 0s - loss: 0.0057

In [40]:
report()

[1. 1. 0. 0. 0.] 	 0 	 [-0.8918855] 	 [0.08596124] 	 [2.2418983 0.        0.        0.       ]
[1. 0. 1. 0. 0.] 	 1 	 [0.945177] 	 [0.96741253] 	 [0.       2.375855 0.       0.      ]
[1. 0. 0. 1. 0.] 	 1 	 [0.9474933] 	 [0.9608007] 	 [0.        0.        2.3816774 0.       ]
[1. 0. 0. 0. 1.] 	 0 	 [-0.9995633] 	 [0.01498006] 	 [0.        0.        0.        2.5125637]
[1. 1. 1. 0. 0.] 	 1 	 [0.8690133] 	 [0.95793664] 	 [0.       2.184405 0.       0.      ]
[1. 1. 0. 1. 0.] 	 1 	 [0.8743397] 	 [0.95072967] 	 [0.       0.       2.197794 0.      ]
[1. 1. 0. 0. 1.] 	 0 	 [-0.99982435] 	 [0.00095742] 	 [2.5132198 0.        0.        2.5132198]
[1. 0. 1. 1. 0.] 	 1 	 [0.99978405] 	 [0.9989416] 	 [0.        2.5131185 2.5131185 0.       ]
[1. 0. 1. 0. 1.] 	 0 	 [-0.89656985] 	 [0.02310733] 	 [0.       0.       0.       2.253673]
[1. 0. 0. 1. 1.] 	 0 	 [-0.8921315] 	 [0.02354094] 	 [0.        0.        0.        2.2425165]
[1. 1. 1. 1. 0.] 	 1 	 [0.99946314] 	 [0.9989392] 	 [0.       2.512312 

In [41]:
def np_present_by_doc(X, human_terms, index, weight):
    np_vector = np.zeros([X.shape[0],])
    
    for i, x in enumerate(X):
        if x[index] == 1:
            np_vector[i] = np.sign(weight)
    
    return np_vector

In [42]:
def load_unigrams(path, X, y):
    word_list = []
    connotation = {}
    
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            word_list.append(line.strip())
            
    for word in word_list:
        pos_count = 0
        neg_count = 0
        for i, doc in enumerate(X):
            if word in doc.lower():
                if (y[i] == 1):
                    pos_count += 1
                else:
                    neg_count += 1
                    
        if pos_count > neg_count:
            connotation[word] = 1
        else:
            connotation[word] = 0
    
    return word_list, connotation

def generate_appearance(X_train_corpus, X_test_corpus, word_list, connotation):
    y_train_agreement = []
    for i in range(len(X_train_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_train_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_train_agreement.append(doc_agreement)
        
    y_test_agreement = []
    for i in range(len(X_test_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_test_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_test_agreement.append(doc_agreement)
        
    return np.array(y_train_agreement), np.array(y_test_agreement)

# 'imdb-unigrams.txt'

### Now try on IMDB data
#### Like the real IMDB

In [43]:
# load data
# make sure that the first shape is the IMDB training data. 

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtrain.pickle')
X_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtest.pickle')
y_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytrain.pickle')
y_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytest.pickle')

In [44]:
# Count vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df = 100)
X_train = cv.fit_transform(X_train_original)
X_test = cv.transform(X_test_original)

In [45]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [48]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_original, X_test_original, 
                                                          word_list, connotation)

In [49]:
y_train_agreement[:,1]

array([0, 0, 0, ..., 0, 0, 0])

In [50]:
len(word_list)

83

In [51]:
# build the combined model
# Combined model
human_terms_len = len(word_list)

base_model = build_base_model(X_train.shape[1])

combined_input_layer = Input(shape=(X_train.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# multiply and pass it into relu
# initialize relu layer

relu_layer = Dense(1, activation='relu', name='relu_layer', use_bias=False, kernel_initializer='ones')

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(relu_layer(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 83)           0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 3641)         0                                            
_________________________________________

In [52]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [53]:
y_train_tanh = y_train_original
y_train_tanh[y_train_tanh == 0] = -1

In [55]:
base_model.fit(X_train, y_train_tanh, batch_size=1, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 137s 5ms/step - loss: 0.4010 - acc: 0.7676
Epoch 2/10
25000/25000 [==============================] - 134s 5ms/step - loss: 0.3367 - acc: 0.8382
Epoch 3/10
 8992/25000 [=========>....................] - ETA: 1:28 - loss: 0.2817 - acc: 0.8644

KeyboardInterrupt: 

In [ ]:
base_model.fit(X_train, y_train_tanh, batch_size=1, epochs=10)

In [ ]:
combined_model.fit([X_train,
                   y_train_agreement])